In [2]:
from pyspark import SparkContext
import pandas as pd
from datetime import datetime

In [3]:
sc = SparkContext()

In [5]:
df = pd.read_excel('Mensuales-20241113-184335.xlsx')

In [6]:
print(df.head())

     CAMBIO      FECHA
0  3.873395 2022-11-01
1  3.826305 2022-12-01
2  3.831490 2023-01-01
3  3.838095 2023-02-01
4  3.777665 2023-03-01


In [7]:
df = df[['CAMBIO', 'FECHA']]

In [8]:
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%d/%m/%Y')

In [9]:
print(df.head())

     CAMBIO      FECHA
0  3.873395 2022-11-01
1  3.826305 2022-12-01
2  3.831490 2023-01-01
3  3.838095 2023-02-01
4  3.777665 2023-03-01


In [11]:
rdd_data = sc.parallelize(df.values.tolist())

In [12]:
#Procesar las filas del RDD
def procesar_datos(row):
    valor = row[0]
    fecha = row[1]
    return (fecha, valor)

# Aplicar la transformación al RDD
rdd_transformado = rdd_data.map(procesar_datos)

In [13]:
rdd_filtrado = rdd_transformado.filter(lambda x: x[1] != '' and x[1] is not None)

In [14]:
#Análisis histórico (promedio del tipo de cambio)
total_cambio = rdd_transformado.map(lambda x: x[1]).sum()  # Sumar todos los cambios
cantidad_filas = rdd_transformado.count()  # Contar las filas

In [15]:
# Calcular el promedio
if cantidad_filas > 0:
    promedio_cambio = total_cambio / cantidad_filas
    print(f"El promedio histórico del tipo de cambio es: S/ {promedio_cambio:.2f}")
else:
    print("No hay datos suficientes para calcular el promedio.")

El promedio histórico del tipo de cambio es: S/ 3.76


In [16]:
#Identificar si el tipo de cambio actual es favorable
ultimo_tipo_cambio = rdd_transformado.takeOrdered(1, key=lambda x: x[0])[-1][1]  # El último tipo de cambio registrado
print(f"Último tipo de cambio registrado: S/ {ultimo_tipo_cambio:.2f}")

Último tipo de cambio registrado: S/ 3.87


In [17]:
# Comparar el tipo de cambio actual con el promedio
if ultimo_tipo_cambio < promedio_cambio:
    print("BUEN CAMBIO")
else:
    print("NO ES BUEN MOMENTO PARA REALIZAR EL CAMBIO")


NO ES BUEN MOMENTO PARA REALIZAR EL CAMBIO
